In [1]:
import cv2
import numpy as np
from scipy.spatial import distance as dist
from collections import OrderedDict

In [20]:
def color_det(image,c):
    colors=OrderedDict({"red":(255,0,0),"green":(0,255,0),"blue":(0,0,255)})
    lab=np.zeros((len(colors),1,3),dtype="uint8")
    color_name=[]
    for (i,(name,rgb)) in enumerate(colors.items()):
        lab[i]=rgb
        color_name.append(name)
    lab=cv2.cvtColor(lab,cv2.COLOR_RGB2LAB)
    
    mask=np.zeros(img.shape[:2],dtype="uint8")
    cv2.drawContours(mask,[c],-1,255,-1)
    mask=cv2.erode(mask,None,iterations=2)
    mean=cv2.mean(image,mask=mask)[:3]
    mindist=(np.inf,None)
    
    for (i,row) in enumerate(lab):
        d=dist.euclidean(row[0],mean)
        if d<mindist[0]:
            mindist=(d,i)
    return color_name[mindist[1]]

In [21]:
def shape_det(c):
    approx=cv2.approxPolyDP(c,0.04*cv2.arcLength(c,True),True)
    x1=approx.ravel()[0]
    y1=approx.ravel()[1]
    if len(approx)==3:
        return "Triangle"
    elif len(approx)==4:
        x1,y1,w,h=cv2.boundingRect(approx)
        ap_rt=float(w)/h
        if ap_rt>=0.95 and ap_rt<=1.05:
            return "Square"
        else:
            return "Rectangle"
    elif len(approx)==5:
        return "Pentagon"
    elif len(approx)==6:
        return "Hexagon"
    else:
        return "Circle"

In [22]:
img=cv2.imread('shapes1.jpg')
blur=cv2.GaussianBlur(img,(5,5),0)
lab=cv2.cvtColor(blur,cv2.COLOR_BGR2LAB)
gray=cv2.cvtColor(blur,cv2.COLOR_BGR2GRAY)
thresh=cv2.threshold(gray,25,255,cv2.THRESH_BINARY)[1]

cnts,hier=cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

for c in cnts:
    approx=cv2.approxPolyDP(c,0.04*cv2.arcLength(c,True),True)
    x1=approx.ravel()[0]
    y1=approx.ravel()[1]
    shapes=shape_det(c)
    colors=color_det(lab,c)
    txt="{},{}".format(colors,shapes)
    cv2.putText(img,txt,(x1,y1),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
    
cv2.imshow('Output',img)
cv2.waitKey(1)

(161.6420233463035, 71.18703877633168, 168.49993291292097)
(128.75606522603738, 201.6090414954262, 178.90772902028368)
(128.69149696320113, 201.55180421579135, 178.89853519113967)
(161.627135348226, 71.1224047306176, 168.76537450722734)
(127.93832853025937, 200.80547550432277, 178.19827089337176)
(0.0, 0.0, 0.0)


-1

In [ ]:
cv2.destroyAllWindows()